In [21]:
from model import *
from data import *
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint 
import segmentation_models as sm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
BACKBONE = 'resnet34' #Pretrained backbone

In [23]:
# Tried data from the same distribution, split data from our satelite images into train and validation sets

def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict=data_gen_args,image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,
                   target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict, validation_split=0.2, rescale=1./255)
    mask_datagen = ImageDataGenerator(**aug_dict, validation_split=0.2, rescale=1./255)
    
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        subset = 'training',
        seed = seed) 
    image_val_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        subset = 'validation',
        seed = seed)
    
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        subset = 'training',
        seed = seed)
    mask_val_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        subset = 'validation',
        seed = seed)
    
        
    return image_generator, image_val_generator, mask_generator, mask_val_generator


In [24]:
image_generator, image_val_generator, mask_generator, mask_val_generator = trainGenerator(8,'dataset','images_satellite','labels_satellite',data_gen_args)

Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.
Found 22 images belonging to 1 classes.
Found 5 images belonging to 1 classes.


In [25]:
def train(image_generator, mask_generator, flag_multi_class = False, num_class = 2):
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)       
        yield (img,mask)
        
def val(image_val_generator, mask_val_generator, flag_multi_class = False, num_class = 2):
    val_generator = zip(image_val_generator, mask_val_generator)
    for (img_val,mask_val) in val_generator:
        img_val,mask_val = adjustData(img_val,mask_val,flag_multi_class,num_class)       
        yield (img_val,mask_val)
        
train_generator = train(image_generator, mask_generator)
val_generator = val(image_val_generator, mask_val_generator)

In [28]:
model = sm.Unet(BACKBONE,encoder_weights='imagenet',input_shape=(256,256,3),encoder_freeze=True)
model.compile(tf.keras.optimizers.Adam(),loss=sm.losses.bce_jaccard_loss,metrics=['accuracy', mean_iou])

In [31]:
model_checkpoint = ModelCheckpoint('pretrained_unet_custom.hdf5', monitor='val_mean_iou',verbose=1, save_best_only=True, mode = 'max')
model.fit_generator(train_generator,steps_per_epoch=100,epochs=2,validation_data=val_generator,validation_steps=100, callbacks=[model_checkpoint])

Epoch 1/2
100/100 [==============================] - ETA: 0s - loss: 0.1024 - accuracy: 0.8410 - mean_iou: 0.9474
Epoch 00001: val_mean_iou improved from -inf to 0.60981, saving model to pretrained_unet_custom.hdf5
100/100 [==============================] - 34s 340ms/step - loss: 0.1024 - accuracy: 0.8410 - mean_iou: 0.9474 - val_loss: 0.8340 - val_accuracy: 0.7743 - val_mean_iou: 0.6098
Epoch 2/2
100/100 [==============================] - ETA: 0s - loss: 0.1025 - accuracy: 0.8412 - mean_iou: 0.9471
Epoch 00002: val_mean_iou improved from 0.60981 to 0.61601, saving model to pretrained_unet_custom.hdf5
100/100 [==============================] - 34s 341ms/step - loss: 0.1025 - accuracy: 0.8412 - mean_iou: 0.9471 - val_loss: 0.8313 - val_accuracy: 0.7767 - val_mean_iou: 0.6160


In [ ]:
# for layer in model.layers:
#     layer.trainable = True
# model.compile(tf.keras.optimizers.Adam(1e-6),loss=sm.losses.bce_jaccard_loss,metrics=['accuracy', mean_iou])
# model.fit_generator(train_generator,steps_per_epoch=500,epochs=10,validation_data=val_generator,validation_steps=100, callbacks=[model_checkpoint])

In [33]:
test_img, _= test('dataset/test_images')
print(test_img.shape)

folder = './dataset/test_labels'
num_img, size = test_img.shape[0], test_img.shape[1]


def generate_labels(folder, num_img, size, predict):
    i=0
    test_label = np.zeros((num_img, size,size,1))
    for files in os.listdir(folder):
        if predict:
            if files.startswith("custom"):
                mask=load_img(os.path.join(folder,files),target_size=(size,size), color_mode="grayscale")
                mask=img_to_array(mask)
                if(np.max(mask) > 1):
                    mask = mask /255
                mask[mask > 0.5] = 1
                mask[mask <= 0.5] = 0
                test_label[i,:,:,:] = mask
                i+=1
        else:
            if not files.startswith("custom"):
                mask=load_img(os.path.join(folder,files),target_size=(size,size), color_mode="grayscale")
                mask=img_to_array(mask)
                if(np.max(mask) > 1):
                    mask = mask /255
                mask[mask > 0.5] = 1
                mask[mask <= 0.5] = 0
                test_label[i,:,:,:] = mask
                i+=1
    return test_label

test_label = generate_labels(folder, num_img, size, False)
print(test_label.shape)

(4, 256, 256, 3)
(4, 256, 256, 1)


In [38]:
model = load_model('pretrained_unet_custom.hdf5', compile = False)
model.compile(tf.keras.optimizers.Adam(1e-6),loss=sm.losses.bce_jaccard_loss,metrics=['accuracy', mean_iou])

score, accuracy, mean_iou= model.evaluate(test_img, test_label)
print('Test score:', score)

1/1 [==============================] - 0s 1ms/step - loss: 3.0581 - accuracy: 0.7702 - mean_iou: 0.6325
Test score: 3.0580530166625977


In [ ]:
# def test2(test_path,target_size=(256,256)):
#     X_test=[]
#     names=[]
#     for filename in os.listdir(test_path):
#         name, ext = os.path.splitext(filename)
#         if ext!=".png" and ext!=".jpg":
#             continue
#         names.append(filename)
#         img=load_img(os.path.join(test_path,filename),target_size=target_size)
#         img=img_to_array(img)/255
#         X_test.append(img.copy())
#     return np.array(X_test),names

In [ ]:
# ### TO DO ####
# import os

# #model=satellite_unet(pretrained='sat_unet_new.hdf5')
# X_test,names=test2("dataset/test_images")
# preds=model.predict(X_test)
# preds=preds>0.5
# results=predToImgs(preds)
# saveResults(os.getcwd()+"/dataset/generator_images",results,names,empty_dir=False)